In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

train = pd.read_csv('../input/playground-series-s4e5/train.csv')
test = pd.read_csv('../input/playground-series-s4e5/test.csv')

trainDropped = train.drop('id', axis=1) 
testDropped = test.drop('id', axis=1) 

In [3]:
target = "FloodProbability"
randomState = 70
n_splits = 5
X = trainDropped.drop(target, axis=1)
y = trainDropped[target]

scaler = StandardScaler()
scaler.fit(X)

X = pd.DataFrame(scaler.transform(X), columns=X.columns)

In [4]:
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=randomState)

kf = KFold(n_splits=n_splits, shuffle=True, random_state=randomState)
scores = []
lgbm_params = {
        'num_leaves': 100, 
        'learning_rate': 0.015257139375608821, 
        'n_estimators': 1000, 
        'subsample_for_bin': 298999, 
        'min_child_samples': 77, 
        'reg_alpha': 2.7240879929233532e-05, 
        'reg_lambda': 9.693029775370852e-10, 
        'colsample_bytree': 0.8107657422421071, 
        'subsample': 0.8727733774586144, 
        'max_depth': 10,
        'objective':'regression',
        'random_state': randomState, 
        'verbose': -1,  # これを指定しないと`No further splits with positive gain, best gain: -inf`というWarningが表示される
        'early_stopping_round': 10,  # ここでearly_stoppingを指定
        'metric': 'rmse',
    }

for tr_idx, va_idx in kf.split(X_train):
    tr_x, va_x = X_train.iloc[tr_idx], X_train.iloc[va_idx]
    tr_y, va_y = y_train.iloc[tr_idx], y_train.iloc[va_idx]
    
    lgbm_model = LGBMRegressor(**lgbm_params)
    
    lgbm_model.fit(tr_x, tr_y,
                   eval_set=[(va_x, va_y)])
    print("aaaaaaaaaaa")
    va_pred = lgbm_model.predict(va_x)
    print("bbbbbbbbbbb")
    score = r2_score(va_y, va_pred)
    scores.append(score)
print(f'r2_score: {np.mean(scores): .4f}')

aaaaaaaaaaa
bbbbbbbbbbb
aaaaaaaaaaa
bbbbbbbbbbb
aaaaaaaaaaa
bbbbbbbbbbb
aaaaaaaaaaa
bbbbbbbbbbb
aaaaaaaaaaa
bbbbbbbbbbb
r2_score:  0.8329


In [5]:
scores

[0.832692118896988,
 0.8325372102794844,
 0.8328158334320993,
 0.8331178394157726,
 0.8331653862656196]

In [6]:
pred = lgbm_model.predict(X_test)
score = r2_score(y_test, pred)
print(f'r2_score: {score: .4f}')

r2_score:  0.8329


In [7]:
submit = test[['id']]
testScaled = pd.DataFrame(scaler.transform(testDropped), columns=testDropped.columns)
submit['FloodProbability'] = lgbm_model.predict(testScaled)
submit
print(submit)

             id  FloodProbability
0       1117957          0.572106
1       1117958          0.455355
2       1117959          0.460769
3       1117960          0.461816
4       1117961          0.459684
...         ...               ...
745300  1863257          0.480732
745301  1863258          0.451351
745302  1863259          0.613848
745303  1863260          0.545372
745304  1863261          0.511784

[745305 rows x 2 columns]


/tmp/ipykernel_18/4201770460.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['FloodProbability'] = lgbm_model.predict(testScaled)


In [8]:
submit.to_csv('submission1.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


## 